In [ ]:
from mpasanalysis import *
import e3sm_res_cmp
%matplotlib inline

In [ ]:
# get paths of restart files, monthly mean output files, processed climatology files and output figures
rst_root, mon_root, climo_root, fig_root = e3sm_res_cmp.load_paths_ocn()
rst_file = rst_root+'/mpaso.rst.0051-01-01_00000.nc'
mon_file = mon_root+'/mpaso.hist.am.timeSeriesStatsMonthly.0041-01-01.nc'

In [ ]:
# flags
save_fig = False

In [ ]:
# load dataset
f_rst = Dataset(rst_file, 'r')
f_mon = Dataset(mon_file, 'r')

In [ ]:
# read grid information
lon = np.degrees(f_rst.variables['lonCell'][:])
lat = np.degrees(f_rst.variables['latCell'][:])
cellArea = f_rst.variables['areaCell'][:]


### SST (degC) and SSS (psu)

In [ ]:
# read data
sst = f_mon.variables['timeMonthly_avg_activeTracers_temperature'][0,:,0]
sss = f_mon.variables['timeMonthly_avg_activeTracers_salinity'][0,:,0]
mpaso_sst = MPASOMap(data=sst, lat=lat, lon=lon, cellarea=cellArea, name='SST', units='degC')
mpaso_sss = MPASOMap(data=sss, lat=lat, lon=lon, cellarea=cellArea, name='SSS', units='psu')


In [ ]:
# plot figure
fig, axarr = plt.subplots(2)
fig.set_size_inches(6, 12)
mpaso_sst.plot(region='LabSea', axis=axarr[0])
mpaso_sss.plot(region='LabSea', axis=axarr[1])

if save_fig:
    figname = fig_root+'/LabSea_sst_sss.png'
    plt.savefig(figname, dpi = 300)

### Sea Surface Potential Density (kg m^{-3})

In [ ]:
# read data
pd = f_mon.variables['timeMonthly_avg_potentialDensity'][0,:,0]
mpaso_pd = MPASOMap(data=pd, lat=lat, lon=lon, cellarea=cellArea, name='Potential Density', units='kg m^{-3}')


In [ ]:
# plot figure
fig = plt.figure()
fig.set_size_inches(6, 6)
mpaso_pd.plot(region='LabSea')

if save_fig:
    figname = fig_root+'/LabSea_pd.png'
    plt.savefig(figname, dpi = 300)

### Sea Surface Current (m s^{-1})

In [ ]:
# read data
velx = f_mon.variables['timeMonthly_avg_velocityZonal'][0,:,0]
vely = f_mon.variables['timeMonthly_avg_velocityMeridional'][0,:,0]
mpaso_velx = MPASOMap(data=velx, lat=lat, lon=lon, cellarea=cellArea, name='Zonal Surface Current', units='m s^{-1}')
mpaso_vely = MPASOMap(data=vely, lat=lat, lon=lon, cellarea=cellArea, name='Meridional Surface Current', units='m s^{-1}')


In [ ]:
# plot figure
fig, axarr = plt.subplots(2)
fig.set_size_inches(6, 12)
levels = np.linspace(-0.4, 0.4, 21)
mpaso_velx.plot(region='LabSea', axis=axarr[0], levels=levels, cmap='RdBu_r')
mpaso_vely.plot(region='LabSea', axis=axarr[1], levels=levels, cmap='RdBu_r')

if save_fig:
    figname = fig_root+'/LabSea_surface_current.png'
    plt.savefig(figname, dpi = 300)

### Mixed Layer Depth (m)

In [ ]:
# read data
mld_d = f_mon.variables['timeMonthly_avg_dThreshMLD'][0,:]
mld_t = f_mon.variables['timeMonthly_avg_tThreshMLD'][0,:]
mpaso_mld_d = MPASOMap(data=mld_d, lat=lat, lon=lon, cellarea=cellArea, name='MLD_rho', units='m')
mpaso_mld_t = MPASOMap(data=mld_t, lat=lat, lon=lon, cellarea=cellArea, name='MLD_temp', units='m')
mpaso_mld_diff = MPASOMap(data=mld_t-mld_d, lat=lat, lon=lon, cellarea=cellArea, name='MLD_temp - MLD_rho', units='m')


In [ ]:
# plot figure
f, axarr = plt.subplots(3)
f.set_size_inches(6, 18)

levels = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 110, 130, 150, 180, 210, 240, 280, 320, 360, 407, 454, 500, 1000, 1500, 2000])
levels_diff = np.linspace(-50, 50, 21)
im0 = mpaso_mld_t.plot(region='LabSea', axis=axarr[0], levels=levels)
im1 = mpaso_mld_d.plot(region='LabSea', axis=axarr[1], levels=levels)
im2 = mpaso_mld_diff.plot(region='LabSea', axis=axarr[2], levels=levels_diff, cmap='RdBu_r')

if save_fig:
    figname = fig_root+'/LabSea_mld.png'
    plt.savefig(figname, dpi = 300)

### Surface Heat Flux (W m$^{-2}$)

In [ ]:
# read data
shf = f_mon.variables['timeMonthly_avg_sensibleHeatFlux'][0,:]
lhf = f_mon.variables['timeMonthly_avg_latentHeatFlux'][0,:]
swf = f_mon.variables['timeMonthly_avg_shortWaveHeatFlux'][0,:]
lwfu = f_mon.variables['timeMonthly_avg_longWaveHeatFluxUp'][0,:]
lwfd = f_mon.variables['timeMonthly_avg_longWaveHeatFluxDown'][0,:]
lwf = lwfu+lwfd
mpaso_swf = MPASOMap(data=swf, lat=lat, lon=lon, cellarea=cellArea, name='SWF', units=r'W m^{-2}')
mpaso_lwf = MPASOMap(data=lwf, lat=lat, lon=lon, cellarea=cellArea, name='LWF', units=r'W m^{-2}')
mpaso_shf = MPASOMap(data=shf, lat=lat, lon=lon, cellarea=cellArea, name='SHF', units=r'W m^{-2}')
mpaso_lhf = MPASOMap(data=lhf, lat=lat, lon=lon, cellarea=cellArea, name='LHF', units=r'W m^{-2}')


In [ ]:
# plot figure
f, axarr = plt.subplots(2,2)
f.set_size_inches(12, 12)
levels = np.linspace(-400, 400, 81)
im0 = mpaso_swf.plot(region='LabSea', axis=axarr[0,0], levels=levels, cmap='RdBu_r')
im1 = mpaso_lwf.plot(region='LabSea', axis=axarr[0,1], levels=levels, cmap='RdBu_r')
im2 = mpaso_shf.plot(region='LabSea', axis=axarr[1,0], levels=levels, cmap='RdBu_r')
im3 = mpaso_lhf.plot(region='LabSea', axis=axarr[1,1], levels=levels, cmap='RdBu_r')

if save_fig:
    figname = fig_root+'/LabSea_heatflux.png'
    plt.savefig(figname, dpi = 300)

### Surface Freshwater  Flux (kg m$^{-2}$ s$^{-1}$)

In [ ]:
# read data
evap = f_mon.variables['timeMonthly_avg_evaporationFlux'][0,:]
rain = f_mon.variables['timeMonthly_avg_rainFlux'][0,:]
snow = f_mon.variables['timeMonthly_avg_snowFlux'][0,:]
si_s = f_mon.variables['timeMonthly_avg_seaIceSalinityFlux'][0,:]
si_f = f_mon.variables['timeMonthly_avg_seaIceFreshWaterFlux'][0,:]
ro_r = f_mon.variables['timeMonthly_avg_riverRunoffFlux'][0,:]
ro_i = f_mon.variables['timeMonthly_avg_iceRunoffFlux'][0,:]
prec = rain+snow
sice = si_s+si_f
roff = ro_r+ro_i
mpaso_evap = MPASOMap(data=evap, lat=lat, lon=lon, cellarea=cellArea, name='Evaporation', units=r'kg m^{-2} s^{-1}')
mpaso_prec = MPASOMap(data=prec, lat=lat, lon=lon, cellarea=cellArea, name='Precipitation', units=r'kg m^{-2} s^{-1}')
mpaso_sice = MPASOMap(data=sice, lat=lat, lon=lon, cellarea=cellArea, name='Sea Ice', units=r'kg m^{-2} s^{-1}')
mpaso_roff = MPASOMap(data=roff, lat=lat, lon=lon, cellarea=cellArea, name='Runoff', units=r'kg m^{-2} s^{-1}')


In [ ]:
# plot figure
f, axarr = plt.subplots(2,2)
f.set_size_inches(12, 12)
levels = np.linspace(-2e-4, 2e-4, 41)
im0 = mpaso_evap.plot(region='LabSea', axis=axarr[0,0], levels=levels, cmap='RdBu_r')
im1 = mpaso_prec.plot(region='LabSea', axis=axarr[0,1], levels=levels, cmap='RdBu_r')
im2 = mpaso_sice.plot(region='LabSea', axis=axarr[1,0], levels=levels, cmap='RdBu_r')
im3 = mpaso_roff.plot(region='LabSea', axis=axarr[1,1], levels=levels, cmap='RdBu_r')

if save_fig:
    figname = fig_root+'/LabSea_freshwaterflux.png'
    plt.savefig(figname, dpi = 300)

### Wind stress (N m^{-2})

In [ ]:
# read data
taux = f_mon.variables['timeMonthly_avg_windStressZonal'][0,:]
tauy = f_mon.variables['timeMonthly_avg_windStressMeridional'][0,:]
mpaso_taux = MPASOMap(data=taux, lat=lat, lon=lon, cellarea=cellArea, name='Zonal Wind Stress', units='N m^{-2}')
mpaso_tauy = MPASOMap(data=tauy, lat=lat, lon=lon, cellarea=cellArea, name='Meridional Wind Stress', units='N m^{-2}')


In [ ]:
# plot figure
fig, axarr = plt.subplots(2)
fig.set_size_inches(6, 12)
levels = np.linspace(-0.4, 0.4, 41)
mpaso_taux.plot(region='LabSea', axis=axarr[0], levels=levels, cmap='RdBu_r')
mpaso_tauy.plot(region='LabSea', axis=axarr[1], levels=levels, cmap='RdBu_r')

if save_fig:
    figname = fig_root+'/LabSea_wind_stress.png'
    plt.savefig(figname, dpi = 300)